In [2]:
# Initial Setup and Imports
import sys
import os

try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="o3")


convisoft_prd = load_artifact("capstone/convisoft_prd.md")


2025-10-02 14:19:58,364 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


In [5]:
# Generate the Pydantic Models for Data Validation
schema_prompt = prompt_enhancer(f"""
You are an expert database architect. Given the following Product Requirements Document (PRD), generate a complete SQL schema for a relational database that fulfills the requirements. Ensure to include tables, columns with appropriate data types, primary keys, foreign keys, and any necessary constraints.
The database should use standard SQL syntax compatible with SQLite.
Create any join tables needed for many-to-many relationships.

PRD:
{convisoft_prd}
""")

print("--- Generating SQL Schema ---")
if convisoft_prd:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'capstone/convisoft_schema.sql', overwrite=True)

2025-10-02 11:07:12,582 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating SQL Schema ---
-- Enable foreign-key enforcement in SQLite
PRAGMA foreign_keys = ON;

--------------------------------------------------------------------
-- 1. Reference / Lookup Tables
--------------------------------------------------------------------
CREATE TABLE naics_codes (
    naics_code        TEXT PRIMARY KEY,               -- 6-digit string
    description       TEXT NOT NULL
);

CREATE TABLE psc_codes (
    psc_code          TEXT PRIMARY KEY,               -- 4-character string
    description       TEXT NOT NULL
);

CREATE TABLE roles (
    role_id           INTEGER PRIMARY KEY,
    role_name         TEXT NOT NULL UNIQUE,           -- e.g., admin, analyst
    description       TEXT
);

--------------------------------------------------------------------
-- 2. Core Master Tables
--------------------------------------------------------------------
CREATE TABLE locations (
    location_id       INTEGER PRIMARY KEY,
    address_line1     TEXT NOT NULL,
    addr

In [7]:
convisoft_schema = load_artifact("capstone/convisoft_schema.sql")
db_code_prompt = f"""
You're a Senior Python Developer tasked with creating SQLAlchemy models.
Using the SQL schema below, generate the following:
    SQLAlchemy Models: For each table in the schema, create a corresponding SQLAlchemy model class.

SQL: {convisoft_schema}
"""

print("--- Generating SQLAlchemy Models ---")
if convisoft_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
    save_artifact(generated_db_code, 'capstone/convisoft_db.py', overwrite=True)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models ---

--- Generated Database Code ---
```python
"""
SQLAlchemy ORM models that mirror the schema supplied in the prompt.

• Compatible with SQLAlchemy ≥ 2.0 (works in 1.4 with future-style engine as well)
• All FK “ON UPDATE / ON DELETE” rules present
• Default / on-update timestamps handled with func.now()
• Helpful indexes & CHECK constraints preserved
"""

from __future__ import annotations

from datetime import date, datetime

from sqlalchemy import (
    CheckConstraint,
    Column,
    Date,
    DateTime,
    Float,
    ForeignKey,
    Index,
    Integer,
    Numeric,
    String,
    Table,
    Text,
    UniqueConstraint,
    func,
)
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship


# --------------------------------------------------------------------------- #
#  Base class
# --------------------------------------------------------------------------- #
class Base(DeclarativeBase):
    """Declarative base for all ORM

In [13]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Load PRD from file
convisoft_prd = load_artifact("capstone/convisoft_prd.md")

llm = ChatOpenAI(model="gpt-4o")

# Tavily search tool (LangChain community tool)
search_tool = TavilySearchResults(max_results=3)

# 1. Extract entities and relationships from PRD
extract_entities_prompt = PromptTemplate.from_template(
    """You are a senior product analyst.
Given the following PRD, extract all relevant entities, attributes, and relationships needed for a database schema.
If any information is missing or ambiguous, use web search tools to find details from referenced websites (sam.gov, gsa.gov, fpds.gov).
PRD:
{prd}
Return a structured list of entities, attributes, and relationships.

{agent_scratchpad}"""
)
extract_entities_chain = LLMChain(llm=llm, prompt=extract_entities_prompt)

# 2. Generate SQL schema from extracted requirements
schema_gen_prompt = PromptTemplate.from_template(
    """You are a database architect.
Given these requirements, generate a complete SQL schema for a relational database (SQLite syntax).
If requirements are unclear, use web search tools to clarify using referenced government sites.
Requirements:
{requirements}
Return only the SQL schema.

{agent_scratchpad}"""
)
schema_gen_chain = LLMChain(llm=llm, prompt=schema_gen_prompt)

# 3. Generate SQLAlchemy models from SQL schema
alchemy_gen_prompt = PromptTemplate.from_template(
    """You are a senior Python developer.
Given the following SQL schema, generate SQLAlchemy ORM model classes (Python code).
If best practices are unclear, use web search tools to check.
SQL Schema:
{schema}
Return only the Python code for the models.

{agent_scratchpad}"""
)
alchemy_gen_chain = LLMChain(llm=llm, prompt=alchemy_gen_prompt)

# Agent 1: Entity extraction agent
extract_agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_tool],
    prompt=extract_entities_prompt
    verbose=True
)
extract_executor = AgentExecutor(agent=extract_agent, tools=[search_tool], verbose=True)

# Step 1: Extract entities/relationships
entities_result = extract_executor.invoke({"prd": convisoft_prd})
entities = entities_result["output"] if "output" in entities_result else entities_result

print("\n--- Entities & Relationships ---\n")
print(entities)

# Agent 2: Schema generation agent
schema_agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_tool],
    prompt=schema_gen_prompt,
    verbose=True
)
schema_executor = AgentExecutor(agent=schema_agent, tools=[search_tool], verbose=True)

# Step 2: Generate SQL schema
schema_result = schema_executor.invoke({"requirements": entities})
schema = schema_result["output"] if "output" in schema_result else schema_result

print("\n--- SQL Schema ---\n")
print(schema)

# Agent 3: SQLAlchemy model generation agent
alchemy_agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_tool],
    prompt=alchemy_gen_prompt,
    verbose=True
)
alchemy_executor = AgentExecutor(agent=alchemy_agent, tools=[search_tool], verbose=True)

# Step 3: Generate SQLAlchemy models
alchemy_result = alchemy_executor.invoke({"schema": schema})
alchemy_models = alchemy_result["output"] if "output" in alchemy_result else alchemy_result

print("\n--- SQLAlchemy Models ---\n")
print(alchemy_models)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current US government contract data schema site:sam.gov'}`


[{'title': 'Fiscal Year 2025 - Forecast of Contract Opportunities', 'url': 'https://home.treasury.gov/policy-issues/small-business-programs/small-and-disadvantaged-business-utilization/forecast-of-contract-opportunities/fiscal-year-2025-forecast-of-contract-opportunities', 'content': 'Alcohol and Tobacco Tax and Trade Bureau (TTB)\n\nBureau of Engraving & Printing (BEP)\n\nFinancial Crimes Enforcement Network (FinCEN)\n\nBureau of the Fiscal Service (BFS)\n\nInternal Revenue Service (IRS)\n\nOffice of the Comptroller of the Currency (OCC)\n\nU.S. Mint\n\nOffice of Inspector General (OIG)\n\nTreasury Inspector General for Tax Administration (TIGTA)\n\nSpecial Inspector General for Pandemic Recovery (SIGPR)\n\n## Budget, Financial Reporting, Planning and Performance\n\nStrategic Plan [...] Note: All projected procurements are subjec